In [1]:
from langchain_google_vertexai import ChatVertexAI

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv,dotenv_values
import os


load_dotenv()
env_vars = dotenv_values(".env")
api_key = env_vars.get("OPENAI_API_KEY")
model_client = OpenAIChatCompletionClient(model='gpt-4o-mini', api_key=api_key)

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat


planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [4]:
# from dotenv import load_dotenv

# from langchain_community.utilities import GoogleSerperAPIWrapper

# from autogen_ext.tools.http import HttpTool


# os.environ['SERPER_API_KEY']='bead05022450578faa7498f4c90d85e534c372e0'


# search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

# def search_web(query:str) ->str:
#     """Search the web for the given query and return the results."""
#     try:
#         results = search_tool_wrapper.run(query)
#         return results
#     except Exception as e:
#         print(f"Error occurred while searching the web: {e}")
#         return "No results found."  

In [5]:
import os
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [6]:
from dotenv import load_dotenv

from langchain_tavily import TavilySearch

from autogen_ext.tools.http import HttpTool

search_tool_wrapper = TavilySearch(topic = "general",max_results = 1,)

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.invoke(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."  

In [7]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [8]:
model_client = OpenAIChatCompletionClient(model='gpt-4o-mini', api_key=api_key)

In [9]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web],
    reflect_on_tool_use=False,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [10]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

In [11]:
data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

In [12]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination

In [13]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [14]:
planning_agent.description

'An agent for planning tasks, this agent should be the first to engage when given a new task.'

In [15]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [16]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [17]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To provide a comprehensive response to your inquiry, we need to fulfill these tasks:

1. Identify the Miami Heat player with the highest points during the 2006-2007 NBA season.
2. Calculate the difference in total rebounds for this player between the 2007-2008 and 2008-2009 seasons. Then, compute the percentage change based on these figures.

Let's start by assigning tasks. 

1. WebSearchAgent: Research which Miami Heat player had the most points during the 2006-2007 season.
2. WebSearchAgent: Find the total rebounds for this player during the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons based on the retrieved data.

O

TaskResult(messages=[TextMessage(id='7d79691e-a4b9-4fb0-901e-3132f2cc9828', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 14, 425379, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='8800c856-32aa-4622-8425-e35b98ee75d0', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=187), metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 21, 410874, tzinfo=datetime.timezone.utc), content="To provide a comprehensive response to your inquiry, we need to fulfill these tasks:\n\n1. Identify the Miami Heat player with the highest points during the 2006-2007 NBA season.\n2. Calculate the difference in total rebounds for this player between the 2007-2008 and 2008-2009 seasons. Then, compute the percenta

In [18]:
# With real web search


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your inquiry, we have already completed the following tasks:

1. Identified the Miami Heat player with the highest points during the 2006-2007 season - Dwyane Wade.
2. Calculated the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons.

Here's the summary:
- **2006-2007 Season:** Dwyane Wade was the player with the highest points for the Miami Heat.
- **Total Rebounds:** 
  - 2007-2008 Season: 4.2 rebounds per game.
  - 2008-2009 Season: 5.0 rebounds per game.
  
The percentage increase in his rebounds from the 2007-2008 season to the 2008-2009 season was 19.05%.

TERMINATE


TaskResult(messages=[TextMessage(id='fea1690f-358e-4793-b826-b63a44afe2e5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 43, 111123, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='0f42f932-c195-42e1-9d7a-34705ed9ce8e', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=1116, completion_tokens=174), metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 49, 355571, tzinfo=datetime.timezone.utc), content="To address your inquiry, we have already completed the following tasks:\n\n1. Identified the Miami Heat player with the highest points during the 2006-2007 season - Dwyane Wade.\n2. Calculated the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons.\n\nHere's the summary:\n- **20

In [19]:
state = await selector_team.save_state()

In [20]:
state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'PlanningAgent': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': "To provide a comprehensive response to your inquiry, we need to fulfill these tasks:\n\n1. Identify the Miami Heat player with the highest points during the 2006-2007 NBA season.\n2. Calculate the difference in total rebounds for this player between the 2007-2008 and 2008-2009 seasons. Then, compute the percentage change based on these figures.\n\nLet's start by assigning tasks. \n\n1. WebSearchAgent: Research which Miami Heat player had the most points during the 2006-2007 season.

# custom selector function

In [21]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)

In [22]:
# With real web search
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- TextMessage (PlanningAgent) ----------
To address your query effectively, we have carried out the necessary tasks and obtained the following information:

1. **Highest Scorer for the Miami Heat in the 2006-2007 Season:** 
   - The player with the most points was Dwyane Wade.

2. **Change in Total Rebounds:**
   - For the 2007-2008 NBA season, Dwyane Wade averaged 4.2 rebounds per game.
   - For the 2008-2009 NBA season, his average was 5.0 rebounds per game.
   - The calculated percentage increase in his total rebounds from the 2007-2008 season to the 2008-2009 season is 19.05%.

This summarizes the information you asked for:

- **Player:** Dwyane Wade
- **Seasons:** 2007-2008 to 2008-2009
- **Percentage Change in Rebounds:** 19.05% increase 

TERMINATE


TaskResult(messages=[TextMessage(id='83320fb1-7a64-4203-86cc-454c1c7b587a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 49, 479409, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='227304f8-4273-4a83-b4e1-d723fd0ecbab', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=1346, completion_tokens=202), metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 53, 784727, tzinfo=datetime.timezone.utc), content='To address your query effectively, we have carried out the necessary tasks and obtained the following information:\n\n1. **Highest Scorer for the Miami Heat in the 2006-2007 Season:** \n   - The player with the most points was Dwyane Wade.\n\n2. **Change in Total Rebounds:**\n   - For the 2007-2008 NBA season, 

In [23]:
await selector_team.reset()

In [24]:
# If Data analyst agent is not selected after web search agent
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False,
    selector_func=my_selector_fun)


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (DataAnalystAgent) ----------
Please provide me with the data for the Miami Heat player's statistics for the 2006-2007 season as well as the total rebounds for the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please provide me with the data for the Miami Heat player's statistics for the 2006-2007 season as well as the total rebounds for the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat players' points for the 2006-2007 season, along with their total rebounds for the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat players' points for the 2006-200

TaskResult(messages=[TextMessage(id='c6ce8bb3-825c-4629-ba34-9b2e6f76f4a1', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 53, 839488, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='b71a74a8-89f0-45f0-8285-8f48c69ffa84', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=45), metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 9, 56, 482034, tzinfo=datetime.timezone.utc), content="Please provide me with the data for the Miami Heat player's statistics for the 2006-2007 season as well as the total rebounds for the 2007-2008 and 2008-2009 seasons.", type='TextMessage'), TextMessage(id='d3dd1a56-49ad-4ccd-afe7-8113494a21e9', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens

In [26]:
from typing import List

In [27]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]

In [28]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    

    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    

    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
        

    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
# await sele.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To answer these questions, we need to gather some specific historical data related to the Miami Heat player's performance stats for the specified seasons. Here's how we will break down the tasks:

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Find the total rebounds for the identified player in the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 seasons.

Let's proceed with these tasks.
---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_3KfGOqZqAaxDhVx3aiRA

TaskResult(messages=[TextMessage(id='18674134-3b0a-4537-8424-2bc236c9ae74', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 11, 54, 779627, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='e0a0b05f-f4f4-4a20-b9cb-fcc095af420d', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=130), metadata={}, created_at=datetime.datetime(2025, 8, 8, 14, 12, 3, 667043, tzinfo=datetime.timezone.utc), content="To answer these questions, we need to gather some specific historical data related to the Miami Heat player's performance stats for the specified seasons. Here's how we will break down the tasks:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchA